# urls
https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html


https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html
    
https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html

In [1]:
urls = [' https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html']

# Import

In [2]:
import requests
from functools import reduce
import json
import csv
import os
from bs4 import BeautifulSoup
import random
import time
import re
from lxml import etree
import lxml.html
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

# Extract movie links

In [3]:
movie_links = []
for url in urls:
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a',href=True):
            movie_links.append(link['href'])
    except Exception as ex:
        print('error is ', ex)

In [4]:
len(movie_links)

30000

In [5]:
movie_links[:5]

['https://en.wikipedia.org/wiki/Z.P.G.',
 'https://en.wikipedia.org/wiki/Zee_and_Co.',
 'https://en.wikipedia.org/wiki/And_Now_the_Screaming_Starts!',
 'https://en.wikipedia.org/wiki/The_Asphyx',
 'https://en.wikipedia.org/wiki/Assassin_(1973_film)']

# Stopwords

In [6]:
tokens_without_sw_intro = []
def rm_stopwords(text):
    new_text = text.lower()
    global tokens_without_sw_intro
    stop_words = set(stopwords.words('english')) 
    text_tokens_intro = word_tokenize(new_text)
    tokens_without_sw_intro = [word for word in text_tokens_intro if not word in stopwords.words()]
   
    return tokens_without_sw_intro

# Punctuation

In [7]:
new_words_intro =[]
def rm_punctuation ():
    global new_words_intro
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    punc_intro = ",".join(tokens_without_sw_intro)
    new_words_intro = tokenizer.tokenize(punc_intro)
    return new_words_intro

# Stemming

In [8]:
# streaming_intro = []
def stemming():
    global streaming_intro
    streaming_intro = []
    ps = PorterStemmer()
    for x in new_words_intro:
        
        streaming_intro.append(ps.stem(x))
    return streaming_intro

# Final normalize

In [9]:
# final_intro = []
def Final_normalize():
    global streaming_intro
    final_intro = []
    for i in streaming_intro:
        if len(i) > 3:
            final_intro.append(i)
    return " ".join(final_intro)

# extract title,intro,plot,table_info and save as tsv

In [101]:
check_list = ['Directed by','Produced by','Written by','Starring','Music by','Release date','Running time','Countries',
              'Language','Budget','Country']
vocabulary = []

for number,t in enumerate(movie_links[:5]):  
    web_response = requests.get(t)
    soup = BeautifulSoup(web_response.text, 'html.parser')
    title = soup.title.text.split('-')
    title = title[0]

    # start intro
    response = requests.get(t) 
    soup = BeautifulSoup(response.text, 'html.parser')
    intro =  soup.find('div' , { "id" : "toc" }).find_all_previous('p')[::-1]
    l=[]
    for i in intro:
        x = i.text.strip()
        if x != '':
            l.append(x)
    y= " ".join(l)
    # remove stopwords
    rm_stopword_intro = rm_stopwords(y)
    #remove punchtuation
    rm_punc_intro=rm_punctuation()
    # stemmer
    steming_intro = stemming()
    # final_normalize
    final_norm_intro = Final_normalize()
    # vocabulary
    vocab_intro = list(set(final_norm_intro.split()))
    vocabulary.extend(vocab_intro)


    # start plot
    plot =  soup.find('span' , { "class" : "mw-editsection-bracket" }).find_next('p')
    final_plot = plot.text.strip()
    # remove stopwords
    rm_stopword = rm_stopwords(final_plot)

    # remove punch
    rm_punc=rm_punctuation()

    # stemmer
    steming = stemming()

    #final_normalize
    final_norm_plot = Final_normalize()
    vocab_plot = list(set(final_norm_plot.split()))
    vocabulary.extend(vocab_plot)

    #table
    table = soup.find('table', class_ = 'infobox vevent')
    d={}
    for x in table.find_all('tr')[2:]:
        if x.th.text.lower() in [l.lower() for l in check_list]:
            for i in x.find_all('th'):
                for y in x.find_all('td'):
                    d[i.text]=y.text.lower()
                    for i in check_list:
                        if i not in d.keys():
                            d[i]= 'NA'

    if d['Countries'] == 'NA' and d['Country'] == 'NA':
        del d['Country']
    if d['Countries'] !='NA' and d['Country'] == 'NA':
        del d['Country']
    if d['Countries'] =='NA' and d['Country'] != 'NA':
        del d['Countries']

    w = d.values()
    d_value = "    ".join(list(w))
    # start save to tsv
 
    name = 'C:\\Users\\Profosor\\Desktop\\tsv\\article_{}.tsv'.format(number)
    with open(name, 'wt', encoding='UTF-8') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow([title])
        tsv_writer.writerow([final_norm_intro])
        tsv_writer.writerow([final_norm_plot])
        tsv_writer.writerow([t])
        tsv_writer.writerow([d_value])


In [102]:
print(vocabulary)

['bomb', 'execut', 'direct', 'michael', 'growth', 'entir', 'world', 'featur', 'dystopian', 'whose', 'concern', 'sell', 'book', 'design', 'bleak', 'scienc', '1972', 'chaplin', 'popul', 'fiction', 'year', 'violat', 'oliv', 'best', 'american', 'earth', 'bound', 'danish', 'paul', 'denmark', 'oppress', 'reflect', 'futur', 'overpopul', 'govern', 'children', 'film', 'campu', 'inspir', 'ehrlich', 'almost', 'zero', 'star', 'geraldin', 'reed', 'short', 'perman', 'babyland', 'surfac', 'entir', 'world', 'visit', 'penalti', 'animatron', 'sever', 'bear', 'common', 'resourc', 'past', 'result', 'color', 'deterr', 'plastic', 'brainwash', 'dome', 'mask', 'coupl', 'need', 'pollut', 'conceiv', 'substitut', 'popul', 'breath', 'yearn', 'extinct', 'newborn', 'bright', 'smog', 'next', 'given', 'thick', 'citi', 'outsid', 'reduc', 'suffoc', 'well', 'earth', 'ultim', 'avail', 'affect', 'anim', 'household', 'cover', 'wear', 'futur', 'overpopul', 'becom', 'govern', 'children', 'life', 'death', 'size', 'decre', 'br

# Inverted index

In [103]:
# way true
def createDictionary(directory):
    wordsAdded = {}
    cwd = os.getcwd()
    os.chdir(directory)
    fileList = os.listdir(directory)
    for file in fileList:
        with open(file, 'r') as f:
            total_word = []
            words = f.read().lower().strip().split('\n')
            for word in words[2].split():
                for i,term in enumerate(vocabulary):
                    if word == term:
                        if word not in wordsAdded.keys():
                            wordsAdded[word] = {}
                            wordsAdded[word]['fileNames'] = []
                            wordsAdded[word]['Term_id'] = []
                            
                        if f.name not in wordsAdded[word]['fileNames'] and i not in wordsAdded[word]['Term_id']:
                            wordsAdded[word]['fileNames'] += [f.name]
                            wordsAdded[word]['Term_id'] += [i]
            for word in words[4].split():
                    for i,term in enumerate(vocabulary):
                        if word == term:
                            if word not in wordsAdded.keys():
                                wordsAdded[word] = {}
                                wordsAdded[word]['fileNames'] = []
                                wordsAdded[word]['Term_id'] = []

                            if f.name not in wordsAdded[word]['fileNames'] and i not in wordsAdded[word]['Term_id']:
                                wordsAdded[word]['fileNames'] += [f.name]
                                wordsAdded[word]['Term_id'] += [i]


    os.chdir(cwd)
    return wordsAdded
def writeToFile(words):
    with open('C:\\Users\\Profosor\\Desktop\\tsv\\index-file.csv', 'w',encoding='UTF-8') as indexFile:

        fieldNames = ['Term_id','word', 'fileNames']

        csvWriter = csv.DictWriter(indexFile, fieldnames= fieldNames)

        csvWriter.writeheader()

        for word, fileDetails in words.items():

            fileNameString = reduce(lambda x, y: x + ", " + y, fileDetails['fileNames'])
            fileTerm_id = reduce(lambda  x, y: x, fileDetails['Term_id'])
            csvWriter.writerow({'word': word, 'fileNames': fileNameString, 'Term_id': fileTerm_id})


def main():
    directory = 'C:\\Users\\Profosor\\Desktop\\tsv'
    writeToFile(createDictionary(directory))
    print("Finished")


main()

Finished


In [104]:
print(vocabulary)

['bomb', 'execut', 'direct', 'michael', 'growth', 'entir', 'world', 'featur', 'dystopian', 'whose', 'concern', 'sell', 'book', 'design', 'bleak', 'scienc', '1972', 'chaplin', 'popul', 'fiction', 'year', 'violat', 'oliv', 'best', 'american', 'earth', 'bound', 'danish', 'paul', 'denmark', 'oppress', 'reflect', 'futur', 'overpopul', 'govern', 'children', 'film', 'campu', 'inspir', 'ehrlich', 'almost', 'zero', 'star', 'geraldin', 'reed', 'short', 'perman', 'babyland', 'surfac', 'entir', 'world', 'visit', 'penalti', 'animatron', 'sever', 'bear', 'common', 'resourc', 'past', 'result', 'color', 'deterr', 'plastic', 'brainwash', 'dome', 'mask', 'coupl', 'need', 'pollut', 'conceiv', 'substitut', 'popul', 'breath', 'yearn', 'extinct', 'newborn', 'bright', 'smog', 'next', 'given', 'thick', 'citi', 'outsid', 'reduc', 'suffoc', 'well', 'earth', 'ultim', 'avail', 'affect', 'anim', 'household', 'cover', 'wear', 'futur', 'overpopul', 'becom', 'govern', 'children', 'life', 'death', 'size', 'decre', 'br

In [105]:
directory = 'C:\\Users\\Profosor\\Desktop\\tsv'
createDictionary(directory)

{'short': {'fileNames': ['article_0.tsv'], 'Term_id': [45]},
 'zero': {'fileNames': ['article_0.tsv'], 'Term_id': [41]},
 'popul': {'fileNames': ['article_0.tsv'], 'Term_id': [18]},
 'growth': {'fileNames': ['article_0.tsv'], 'Term_id': [4]},
 '1972': {'fileNames': ['article_0.tsv', 'article_1.tsv', 'article_3.tsv'],
  'Term_id': [16, 144, 324]},
 'danish': {'fileNames': ['article_0.tsv'], 'Term_id': [27]},
 'american': {'fileNames': ['article_0.tsv'], 'Term_id': [24]},
 'dystopian': {'fileNames': ['article_0.tsv'], 'Term_id': [8]},
 'scienc': {'fileNames': ['article_0.tsv', 'article_3.tsv'],
  'Term_id': [15, 322]},
 'fiction': {'fileNames': ['article_0.tsv', 'article_3.tsv'],
  'Term_id': [19, 316]},
 'film': {'fileNames': ['article_0.tsv',
   'article_1.tsv',
   'article_2.tsv',
   'article_3.tsv',
   'article_4.tsv'],
  'Term_id': [36, 162, 243, 301, 317]},
 'direct': {'fileNames': ['article_0.tsv',
   'article_1.tsv',
   'article_2.tsv',
   'article_3.tsv',
   'article_4.tsv'],
  

In [106]:
for l,t in enumerate(vocabulary):
    print(l,t)

0 bomb
1 execut
2 direct
3 michael
4 growth
5 entir
6 world
7 featur
8 dystopian
9 whose
10 concern
11 sell
12 book
13 design
14 bleak
15 scienc
16 1972
17 chaplin
18 popul
19 fiction
20 year
21 violat
22 oliv
23 best
24 american
25 earth
26 bound
27 danish
28 paul
29 denmark
30 oppress
31 reflect
32 futur
33 overpopul
34 govern
35 children
36 film
37 campu
38 inspir
39 ehrlich
40 almost
41 zero
42 star
43 geraldin
44 reed
45 short
46 perman
47 babyland
48 surfac
49 entir
50 world
51 visit
52 penalti
53 animatron
54 sever
55 bear
56 common
57 resourc
58 past
59 result
60 color
61 deterr
62 plastic
63 brainwash
64 dome
65 mask
66 coupl
67 need
68 pollut
69 conceiv
70 substitut
71 popul
72 breath
73 yearn
74 extinct
75 newborn
76 bright
77 smog
78 next
79 given
80 thick
81 citi
82 outsid
83 reduc
84 suffoc
85 well
86 earth
87 ultim
88 avail
89 affect
90 anim
91 household
92 cover
93 wear
94 futur
95 overpopul
96 becom
97 govern
98 children
99 life
100 death
101 size
102 decre
103 break
1

In [1]:
import pandas as pd
df = pd.read_csv('C://Users//Profosor//Desktop//tsv/index-file.csv')

In [2]:
df.shape

(321, 3)

In [4]:
df.head()

,Term_id,word,fileNames
0,45,short,article_0.tsv
1,41,zero,article_0.tsv
2,18,popul,article_0.tsv
3,4,growth,article_0.tsv
4,16,1972,"article_0.tsv, article_1.tsv, article_3.tsv"


In [110]:
df[df['word']=='secret']

,Term_id,word,fileNames
320,390,secret,article_4.tsv


In [111]:
df2 = df.copy()
# df2.drop('word',axis=0,inplace=True)
t_id = df2['Term_id']
f_name = [x.split(',') for x in df2['fileNames']]
# l=[x for x in df2['Term_id']]
# for i in f_name:
#     new = [x.strip() for x in i]
#     l.append(new)
    
d= dict(zip(t_id,f_name))
d

{45: ['article_0.tsv'],
 41: ['article_0.tsv'],
 18: ['article_0.tsv'],
 4: ['article_0.tsv'],
 16: ['article_0.tsv', ' article_1.tsv', ' article_3.tsv'],
 27: ['article_0.tsv'],
 24: ['article_0.tsv'],
 8: ['article_0.tsv'],
 15: ['article_0.tsv', ' article_3.tsv'],
 19: ['article_0.tsv', ' article_3.tsv'],
 36: ['article_0.tsv',
  ' article_1.tsv',
  ' article_2.tsv',
  ' article_3.tsv',
  ' article_4.tsv'],
 2: ['article_0.tsv',
  ' article_1.tsv',
  ' article_2.tsv',
  ' article_3.tsv',
  ' article_4.tsv'],
 3: ['article_0.tsv', ' article_1.tsv'],
 37: ['article_0.tsv'],
 42: ['article_0.tsv',
  ' article_1.tsv',
  ' article_2.tsv',
  ' article_3.tsv',
  ' article_4.tsv'],
 22: ['article_0.tsv'],
 44: ['article_0.tsv'],
 43: ['article_0.tsv'],
 17: ['article_0.tsv'],
 38: ['article_0.tsv'],
 23: ['article_0.tsv', ' article_2.tsv'],
 11: ['article_0.tsv'],
 12: ['article_0.tsv'],
 0: ['article_0.tsv'],
 28: ['article_0.tsv'],
 39: ['article_0.tsv'],
 10: ['article_0.tsv', ' article_

In [ ]:
with open('C:\\Users\\Profosor\\Desktop\\tsv\\d.json', 'w') as f:
    json.dump(d, f,indent=4) 

In [ ]:
with open('C:\\Users\\Profosor\\Desktop\\tsv\\d.json', 'r') as f:
    data =  json.load(f)

# query

In [223]:
query = ['secret','popul','whose']
# query = [x.lower() for x in query]

title = []
intro = []
url = []
for i,term in enumerate(list(set(vocabulary))):
    os.chdir('C:/Users/Profosor/Desktop/tsv')
    for q in query:
        if q == term:
            file_list = d[i]
            for j in file_list:
                li=[]
                with open(j,'r') as f:
                    data = f.read().strip().split('\n')
                    for i in data: 
                        if i!= '':
                            li.append(i)
                title.append(li[0].strip())
                intro.append(li[1])
                url.append(li[3])

                        
dict = {'title':title,'intro':intro,'url':url}

df = pd.DataFrame(dict) 

    
df
    



,title,intro,url
0,Z.P.G.,short zero popul growth 1972 danish american d...,https://en.wikipedia.org/wiki/Z.P.G.
1,Zee and Co.,known compani 1972 british drama film direct b...,https://en.wikipedia.org/wiki/Zee_and_Co.
2,And Now the Screaming Starts!,scream start 1973 british gothic horror film d...,https://en.wikipedia.org/wiki/And_Now_the_Scre...
